# 3. Modules

In [1]:
import torch
import torch.nn as nn

## An Example

In [13]:
conv_1d = torch.nn.Conv1d(in_channels=5, out_channels=2, kernel_size=3)
print(conv_1d)

Conv1d(5, 2, kernel_size=(3,), stride=(1,))


In [14]:
print(conv_1d.weight)

Parameter containing:
tensor([[[-8.7194e-02, -9.0729e-02,  1.3348e-01],
         [ 2.2500e-01, -1.3804e-01,  2.0608e-01],
         [-1.0680e-01, -1.8828e-01, -4.2253e-03],
         [ 1.0990e-01, -1.8608e-01,  1.1721e-02],
         [-2.1816e-01, -2.8256e-03, -9.6215e-02]],

        [[ 1.8073e-02, -2.4290e-01,  8.2565e-03],
         [-1.0163e-01, -1.7370e-01, -2.6027e-03],
         [ 1.2012e-01,  7.9548e-02,  2.5538e-01],
         [-2.1925e-01, -2.0289e-01, -1.4813e-01],
         [ 9.1355e-02,  1.9734e-01, -1.4195e-04]]], requires_grad=True)


In [21]:
for name, tensor in conv_1d.named_parameters():
    print("{:6s}  -  {}".format(name, tensor.shape))

weight  -  torch.Size([2, 5, 3])
bias    -  torch.Size([2])


---
## Basic Usage

In [23]:
# Each instance of a module has its own parameters initialised randomly

linear_regression_model = torch.nn.Linear(5, 2)  # 5 input dimensions, 2 output dimensions

linear_regression_model.weight

Parameter containing:
tensor([[-0.0221,  0.3967, -0.3527, -0.1389, -0.1864],
        [-0.3002, -0.4203, -0.2064, -0.0009, -0.1193]], requires_grad=True)

In [24]:
type(linear_regression_model.weight)

torch.nn.parameter.Parameter

A `Parameter` is a Tensor which is automatically added to the list of parameters when used within a model.

In [31]:
# Pytorch Modules operate on batches. It allows to process multiple datapoints in parallel
batch_size = 3

x = torch.randn(batch_size, 5) # 3 samples with 5 features each

print(x, "\n\n", x.shape)

tensor([[-0.5346,  0.6289,  0.0679,  1.3704, -0.1446],
        [ 1.7458, -0.2663,  0.4745, -0.8237,  0.3194],
        [ 0.1725,  0.3220,  0.8263, -1.8113, -0.3454]]) 

 torch.Size([3, 5])


In [32]:
predicted_y = linear_regression_model(x)  # Note, you do not call forward
print(predicted_y, "\n\n", predicted_y.shape)

tensor([[-0.5470,  0.1475],
        [-0.0658, -1.0979],
        [ 0.5978, -0.6469]], grad_fn=<AddmmBackward>) 

 torch.Size([3, 2])


---

## Composing modules with `torch.nn.Sequential`


In [33]:
neural_net = torch.nn.Sequential(
    torch.nn.Linear(5, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(10, 2),
)

# Run the model:
neural_net(x)

tensor([[-0.5772,  0.0131],
        [-0.1462, -0.0916],
        [-0.2687, -0.2147]], grad_fn=<AddmmBackward>)

---

## Building Custom modules

In [10]:
import torch.nn.functional as F

class MyNeuralNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MyNeuralNetwork, self).__init__()
        
        self.linear_1 = torch.nn.Linear(input_size, hidden_size)
        self.linear_2 = torch.nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = F.relu(self.linear_1(x))
        out = self.linear_2(out)
        return out

In [11]:
x = torch.rand(5, 10)  # the first dimension is reserved for the 'batch_size'

model = MyNeuralNetwork(input_size=10, hidden_size=5,  num_classes=2)

out = model(x)  # this calls model.forward(x)

print(out.shape)

torch.Size([5, 2])


---

## Moving your model to GPU

In [20]:
model.cuda()  # No need to assign it

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [21]:
device = torch.device("cuda:0")
model.to(device)  # No need to assign it

AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

---

## Storing and loading models

### The easy way

In [12]:
torch.save(model, "my_model.pt")

/home/evann/.local/lib/python3.6/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type MyNeuralNetwork. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [13]:
my_model_loaded = torch.load("my_model.pt")

In [14]:
print(model.linear_2.bias)
print(my_model_loaded.linear_2.bias)

Parameter containing:
tensor([ 0.0374, -0.3688], requires_grad=True)
Parameter containing:
tensor([ 0.0374, -0.3688], requires_grad=True)


### The recommended way

In [16]:
torch.save(model.state_dict(), "my_model_state_dict.pt")

In [17]:
my_model_loaded = MyNeuralNetwork(10, 5, 2)
my_model_loaded.load_state_dict(torch.load("my_model_state_dict.pt"))

<All keys matched successfully>

In [18]:
print(model.linear_2.bias)
print(my_model_loaded.linear_2.bias)

Parameter containing:
tensor([ 0.0374, -0.3688], requires_grad=True)
Parameter containing:
tensor([ 0.0374, -0.3688], requires_grad=True)


# TODO: Add losses here

## Building our training loop (3 / 5)

In [35]:
# INITIALIZATION

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, RandomCrop
from torchvision.datasets import ImageFolder

device = torch.device("cpu")

transform = Compose((RandomCrop((50, 50)), ToTensor()))
dataset = ImageFolder(root="alien-vs-predator/", transform=transform)
loader = DataLoader(dataset, batch_size=5, shuffle=True)

model = torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(7500, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 2),
)
model.to(device)

loss_fn = nn.CrossEntropyLoss()

In [37]:
# TRAINING LOOP

for samples, labels in loader:
    samples = samples.to(device)
    labels = labels.to(device)
    predictions = model(samples)
    loss = loss_fn(predictions, labels)
    # compute gradients
    # update model parameters